# Problema del **flujo máximo a coste mínimo** - Ruymán García Martín

In [9]:
import Pkg
Pkg.add("JuMP")
Pkg.add("GLPK")
using JuMP, GLPK, Random, Test

   Resolving package versions...
  No Changes to `C:\Users\Ruymán\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Ruymán\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Ruymán\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Ruymán\.julia\environments\v1.11\Manifest.toml`


# Problema del flujo máximo a coste mínimo




El problema de flujo máximo a coste mínimo en un grafo dirigido es una variante del problema clásico de flujo máximo en un grafo. En este problema, se busca encontrar la cantidad máxima de flujo que puede enviarse desde un nodo fuente (s) a un nodo sumidero (t) en un grafo ponderado dirigido, minimizando el costo total asociado al envío del flujo a través de las aristas.

## Modelo Matemático

### Variables de decisión
* $V \rightarrow$ Conjunto de nodos.
* $E \rightarrow$ Conjunto de aristas.
*   $x_{ij} \rightarrow$ La cantidad de flujo enviado a lo largo de la arista (i, j).
*   $f \rightarrow$ El valor del flujo total desde el nodo fuente $(s)$ al nodo sumidero $(t)$.

### Parámetros
* $c_{ij}\rightarrow$ El costo por unidad de flujo a lo largo de la arista (i, j).
* $u_{ij}\rightarrow$ La capacidad máxima de flujo a lo largo de la arista (i, j).
* $s\rightarrow$ El nodo fuente.
* $t\rightarrow$ El nodo sumidero.

### Función Objetivo

$$
    \begin{array}{ccc}
    minimizar  & \sum_{i,j} c_{ij} x_{ij}\\
    \end{array}
$$

### Restricciones

$$
    \begin{array}{ccc}
    & \sum_{j:(i,j) \in E} x_{ij} - \sum_{j:(j,i) \in E} x_{ji} = 0 & \forall i \in V - \{s,t\}\\
    & \sum_{j:(s,j) \in E} x_{sj} - \sum_{j:(j,s) \in E} x_{js} = f\\
    & \sum_{j:(t,j) \in E} x_{tj} - \sum_{j:(j,t) \in E} x_{jt} = -f\\
    & x_{ij} \leq u_{ij} & \forall(i,j) \in E\\
    & x_{ij} \geq 0\\
    & f \geq 0
    \end{array}
$$

## Modelo en julia

Importación de las bibliotecas necesarias:

In [1]:
using JuMP
using GLPK

Definimos los datos del problema:

Utilizo datos ya definidos, porque en la mayoría de casos con valores aleatorios no suele encontrar una solución óptima. De todas formas, aqui dejo las dos versiones para poder probarlo.

In [2]:
V = 1:5  # Conjunto de nodos
E = [(1,2), (1,3), (2,4), (3,4), (3,5), (4,5)]  # Conjunto de arcos

c = Dict(zip(E, [3, 9, 7, 2, 10, 6])) # Costo de los arcos
u = Dict(zip(E, [4, 10, 3, 6, 3, 10])) # Capacidad de los arcos

s = 1  # Nodo fuente
t = 5  # Nodo sumidero
f = 10  # Flujo deseado

10

In [3]:
V = 1:5  # Conjunto de nodos
E = [(1,2), (1,3), (2,4), (3,4), (3,5), (4,5)]  # Conjunto de arcos

c = Dict((i,j) => rand(1:10) for (i,j) in E)  # Costos de los arcos
u = Dict((i,j) => rand(1:10) for (i,j) in E)  # Capacidad de los arcos

s = 1  # Nodo fuente
t = 5  # Nodo sumidero
f = 10  # Flujo deseado

10

Creación del modelo, definición de las variables de decisión y definición de restricciones:

In [4]:
# Crear el modelo de optimización
model = Model(GLPK.Optimizer)

# Definir las variables de decisión
@variable(model, x[i=V, j=V] >= 0)

# Definir la función objetivo
@objective(model, Min, sum(c[i,j] * x[i,j] for (i,j) in E))

# Restricciones de capacidad
for (i, j) in E
    @constraint(model, x[i, j] <= u[i, j])
end

# Restricciones de flujo
@constraint(model, [i in setdiff(V, [s, t])], sum(x[i,j] for j in V if (i,j) in E) - sum(x[j,i] for j in V if (j,i) in E) == 0)
@constraint(model, sum(x[s,j] for j in V if (s,j) in E) - sum(x[j,s] for j in V if (j,s) in E) == f)
@constraint(model, sum(x[t,j] for j in V if (t,j) in E) - sum(x[j,t] for j in V if (j,t) in E) == -f)

-x[3,5] - x[4,5] == -10

Muestra de resultados:

In [5]:
# Resolver el modelo
optimize!(model)

# Imprimir resultados
if termination_status(model) == MOI.OPTIMAL
    println("Solución óptima encontrada")
    for (i,j) in E
        println("x[", i, ",", j, "] = ", value(x[i,j]))
    end
    println("\nCosto mínimo = ", objective_value(model))
else
    println("El problema no tiene solución óptima.")
end

Solución óptima encontrada
x[1,2] = 3.0
x[1,3] = 7.0
x[2,4] = 3.0
x[3,4] = 5.0
x[3,5] = 2.0
x[4,5] = 8.0

Costo mínimo = 115.0
